In [21]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix


In [3]:
user_profiles = pd.read_table('lastfm-dataset-360K/usersha1-profile.tsv',
                              names = ['users', 'gender', 'age', 'location', 'signup'],
                             header=None)
user_history = pd.read_table('lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',
                            names = ['users', 'MBID', 'artist-name', 'plays'],
                            header=None)

In [4]:
user_profiles.head()

,users,gender,age,location,signup
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007"
1,00001411dc427966b17297bf4d69e7e193135d89,f,NaN,Canada,"Dec 4, 2007"
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,NaN,NaN,Germany,"Sep 1, 2006"
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,m,19.0,Mexico,"Apr 28, 2008"
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,m,28.0,United States,"Jan 27, 2006"


In [5]:
user_history.head()

,users,MBID,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [6]:
user_data = user_history.dropna(axis = 0, subset = ['artist-name'])

In [7]:
user_data.head()

,users,MBID,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [8]:
gpby = user_data.groupby('artist-name')['plays'].sum().reset_index().rename(columns = {'plays': 'total_artist_plays'})[['artist-name', 'total_artist_plays']]


In [9]:
gpby.describe()

,total_artist_plays
count,2.923640e+05
mean,1.290704e+04
std,1.859813e+05
min,1.000000e+00
25%,5.300000e+01
50%,2.080000e+02
75%,1.048000e+03
max,3.046683e+07


In [12]:
gpby[:10]

,artist-name,total_artist_plays
0,04)],6
1,2,1606
2,58725ab=>,23
3,80lİ yillarin tÜrkÇe sÖzlÜ aŞk Şarkilari,70
4,amy winehouse,23
5,cours de la somme,9
6,fatboy slim,16
7,kanye west,56
8,mala rodriguez,24
9,mohamed lamine,113


In [13]:
popularity_threshold = 40000
user_data_with_artist_plays = user_data.merge(gpby, left_on = 'artist-name', right_on = 'artist-name', how = 'left')
user_data_with_artist_plays

,users,MBID,artist-name,plays,total_artist_plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137,25651
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099,3704875
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897,180391
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717,410725
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706,90498
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691,13547741
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545,40139
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507,1004743
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424,58790
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403,61171


In [17]:
user_data_popular_artists = user_data_with_artist_plays.query('total_artist_plays >= @popularity_threshold')


In [27]:
combined = user_data_popular_artists.merge(user_profiles, left_on = 'users', right_on = 'users', how = 'left')
usa_data = combined.query('location == \'United States\'')

In [19]:
if not usa_data[usa_data.duplicated(['users', 'artist-name'])].empty:
    initial_rows = usa_data.shape[0]
    
    print('Initial dataframe shape {0}'.format(usa_data.shape))
    usa_data = usa_data.drop_duplicates(['users', 'artist-name'])
    current_rows = usa_data.shape[0]
    print('New dataframe shape {0}'.format(usa_data.shape))
    print('Removed {0} rows'.format(initial_rows - current_rows))

Initial dataframe shape (2788019, 9)
New dataframe shape (2788013, 9)
Removed 6 rows


In [22]:
wide_artist_data = usa_data.pivot(index = 'artist-name', columns = 'users', values = 'plays').fillna(0)
wide_artist_data_sparse = csr_matrix(wide_artist_data.values)

In [23]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(wide_artist_data_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [39]:
query_index = np.random.choice(wide_artist_data.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(wide_artist_data.iloc[query_index, :].reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(wide_artist_data.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, wide_artist_data.index[indices.flatten()[i]], distances.flatten()[i]))

5054
Recommendations for maroon 5:

1: gavin degraw, with distance of 0.6977651789588328:
2: jay brannan, with distance of 0.7125253320808984:
3: eagle-eye cherry, with distance of 0.7222817328284963:
4: justin timberlake, with distance of 0.7297560502414842:
5: snow patrol, with distance of 0.7305174913486907:


In [44]:
wide_artist_data.shape[0]

9127

In [45]:
q = 4
distances, indices = model_knn.kneighbors(wide_artist_data.iloc[q, :].reshape(1, -1), n_neighbors = 6)


In [48]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(wide_artist_data.index[q]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, wide_artist_data.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for *nsync:

1: justin timberlake, with distance of 0.6631868052912431:
2: m2m, with distance of 0.7173464456035864:
3: backstreet boys, with distance of 0.7208751556780406:
4: 1tym, with distance of 0.7294299581260729:
5: 王力宏, with distance of 0.7371154396238719:
